Ceren Yilmaz Gulten Exercise 5

In [31]:
import numpy as np 
import pandas as pd 

In [82]:
# 1. Pandas Dataframe 
!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P  data/
df = pd.read_csv(r"data/data_000637.txt", sep=",") # reading csv file 
df = pd.DataFrame(df)

--2022-12-02 22:45:29--  https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt
Resolving www.dropbox.com (www.dropbox.com)... 162.125.69.18
Connecting to www.dropbox.com (www.dropbox.com)|162.125.69.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/xvjzaxzz3ysphme/data_000637.txt [following]
--2022-12-02 22:45:30--  https://www.dropbox.com/s/raw/xvjzaxzz3ysphme/data_000637.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc1add74b2b0be11853ebcc5b201.dl.dropboxusercontent.com/cd/0/inline/Bx4lV7WfMvvMz_ZZsC-lp9lF10a-5-UdmO26dqEpibyma41FWTrR5djC2OvE1BrwasXzKnRSsgecud4GhTAXG6iG_0X5jD3zYZZZVQepOkHarzyRXNiZNYu3mdJpVT5_LsfI_b_XJSVBafoshKavPjdZkyX78AZ7xIxrwf_31toxdw/file# [following]
--2022-12-02 22:45:30--  https://uc1add74b2b0be11853ebcc5b201.dl.dropboxusercontent.com/cd/0/inline/Bx4lV7WfMvvMz_ZZsC-lp9lF10a-5-UdmO26dqEpibyma41FWTrR5djC2OvE1BrwasXzKnRSsgecud4GhTAXG6iG_0X5jD3zY

In [83]:
# 1.
row = len(df.axes[0])
print("Number of rows:\t",row)
print("Dataframe for N=15000 rows:\n",df.head(15000)) #N=15000 bigger than 10k 
print("Dataframe for maximum number of rows:\n",df.head(row)) # N=row number(1310720)
df = df.head(15000)

Number of rows:	 1310720
Dataframe for N=15000 rows:
        HEAD  FPGA  TDC_CHANNEL   ORBIT_CNT  BX_COUNTER  TDC_MEAS
0         1     0          123  3869200167        2374        26
1         1     0          124  3869200167        2374        27
2         1     0           63  3869200167        2553        28
3         1     0           64  3869200167        2558        19
4         1     0           64  3869200167        2760        25
...     ...   ...          ...         ...         ...       ...
14995     1     1            4  3869200316        3399         9
14996     1     1           17  3869200316        3400        15
14997     1     1           10  3869200316        3530        16
14998     1     1            8  3869200316        3533        18
14999     1     0          139  3869200316        3539         0

[15000 rows x 6 columns]
Dataframe for maximum number of rows:
          HEAD  FPGA  TDC_CHANNEL   ORBIT_CNT  BX_COUNTER  TDC_MEAS
0           1     0          123  

In [84]:
# 2. 
data = df.loc[df['BX_COUNTER'] == 0] # values where BX_Counter is equals 0 
print(data)
bx_counter = df.head(2894) # first index of BX_Counter is zero is 2894 we can get the data until 2894th line 
x = bx_counter['BX_COUNTER'].max() # get the maximum value from the selected dataset 
print('x=',x)

       HEAD  FPGA  TDC_CHANNEL   ORBIT_CNT  BX_COUNTER  TDC_MEAS
2894      1     0           46  3869200196           0         1
8000      1     0           63  3869200247           0        19
13797     1     1           16  3869200305           0        14
14397     1     0           59  3869200311           0        14
14398     1     1            6  3869200311           0        17
14400     1     0           62  3869200311           0        26
14404     1     1           13  3869200311           0        27
x= 3563


In [128]:
# 3. 
bx = np.array(df['BX_COUNTER'])
orbit = np.array(df['ORBIT_CNT'])
ns  = (bx+orbit*x)*25
df['TIME'] = ns
ns_series = pd.to_datetime(ns,unit='ns')
df['TIME_SERIES'] = ns_series
print(df)

       HEAD  FPGA  TDC_CHANNEL   ORBIT_CNT  BX_COUNTER  TDC_MEAS  \
0         1     0          123  3869200167        2374        26   
1         1     0          124  3869200167        2374        27   
2         1     0           63  3869200167        2553        28   
3         1     0           64  3869200167        2558        19   
4         1     0           64  3869200167        2760        25   
...     ...   ...          ...         ...         ...       ...   
14995     1     1            4  3869200316        3399         9   
14996     1     1           17  3869200316        3400        15   
14997     1     1           10  3869200316        3530        16   
14998     1     1            8  3869200316        3533        18   
14999     1     0          139  3869200316        3539         0   

                  TIME                   TIME_SERIES  
0      344649004934875 1970-01-04 23:44:09.004934875  
1      344649004934875 1970-01-04 23:44:09.004934875  
2      34464900493

In [79]:
#4. 


In [74]:
# 5. 
print('FPGA == 1\n',df[df['FPGA'] == 1].groupby(['TDC_CHANNEL']).count()['FPGA'].nlargest(3))
print('FPGA == 0\n',df[df['FPGA'] == 0].groupby(['TDC_CHANNEL']).count()['FPGA'].nlargest(3))


FPGA == 1
 TDC_CHANNEL
139    389
2      363
1      338
Name: FPGA, dtype: int64
FPGA == 0
 TDC_CHANNEL
139    879
63     743
64     735
Name: FPGA, dtype: int64


In [85]:
# 6. 

nonempty_orbit = df.loc[df['TDC_MEAS'] != 0] # TDC_MEAS is shows if measurement is done  
print("Number of non empty orbit: " , nonempty_orbit.shape[0])

Number of non empty orbit:  13311


In [86]:
# 7. 

tdc_139 = df.loc[df['TDC_CHANNEL']==139 ] # the data where TDC_CHANNEL==139
uniq=tdc_139['ORBIT_CNT'].unique() # take the unique values for ORBIT_CNT from defined dataset
print("Number of unique orbit for TDC_CHANNEL == 139: ",uniq.shape[0]) # shape for count the number


Number of unique orbit for TDC_CHANNEL == 139:  150


In [124]:
# 8. 
fpga_zero = df.loc[df['FPGA'] == 0]
fpga_one = df.loc[df['FPGA'] == 1]
fpga_zero_tdc = pd.Series(fpga_zero['TDC_MEAS'].values, index= fpga_zero['TDC_CHANNEL'])
fpga_one_tdc = pd.Series(fpga_one['TDC_MEAS'].values, index= fpga_one['TDC_CHANNEL'])
print(fpga_zero_tdc)
print(fpga_one_tdc)

TDC_CHANNEL
123    26
124    27
63     28
64     19
64     25
       ..
139     0
61     26
60     23
64     18
139     0
Length: 10470, dtype: int64
TDC_CHANNEL
7       4
6      18
139     0
8      14
5       4
       ..
19     11
4       9
17     15
10     16
8      18
Length: 4530, dtype: int64
